In [35]:
from pyspark.sql import SparkSession
import pyspark

spark = SparkSession.builder.getOrCreate()

In [36]:
df = spark.read.format("csv").option("header", "true").load('Berlin_housing_data.csv')
df

DataFrame[ID: string, Price: string, Housing Type: string, Size: string, Note: string, Provider Type: string, Address: string, Rent Price Info: string, Listing Date: string]

In [37]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn('Address', regexp_replace('Address', 'ß', 'ss'))
df = df.withColumn('Address', regexp_replace('Address', 'ö', 'oe'))
df = df.withColumn('Address', regexp_replace('Address', 'ü', 'ue'))
df = df.withColumn('Address', regexp_replace('Address', 'ä', 'ae'))
df.select('Address').show(n = 20)

+--------------------+
|             Address|
+--------------------+
|Sankt Wolfgang-St...|
|Sankt Wolfgang-St...|
| Alfred-Jung-Strasse|
|         Rhinstrasse|
|        Ebertstrasse|
| Alfred-Jung-Strasse|
|     Rathenaustrasse|
|Osnabruecker Strasse|
|   Monumentenstrasse|
|     Turiner Strasse|
|     Turiner Strasse|
|     Turiner Strasse|
|     Turiner Strasse|
|Reinickendorfer S...|
|Nazarethkirchstrasse|
|    Schlueterstrasse|
|   Zechliner Strasse|
|Nazarethkirchstrasse|
|     Kottbusser Damm|
|Kaiser-Friedrich-...|
+--------------------+
only showing top 20 rows



In [38]:
from pyspark.sql.functions import trim

df = df.withColumn('Housing Type', trim(df['Housing Type']))
df.select('Housing Type').distinct().collect()

[Row(Housing Type='Studio'),
 Row(Housing Type='Mehrbettzimmer'),
 Row(Housing Type='Haus'),
 Row(Housing Type='Wohnung'),
 Row(Housing Type='WG-Zimmer')]

In [39]:
df.select('Note').distinct().collect()

[Row(Note='7  Mitbewohner(innen)'),
 Row(Note='8  Mitbewohner(innen)'),
 Row(Note='1 Mitbewohner(in)'),
 Row(Note='12  Mitbewohner(innen)'),
 Row(Note='3 Schlafzimmer'),
 Row(Note='4 Schlafzimmer'),
 Row(Note='11  Mitbewohner(innen)'),
 Row(Note='Max. 6 Personen'),
 Row(Note='2  Mitbewohner(innen)'),
 Row(Note='5 Schlafzimmer'),
 Row(Note='9  Mitbewohner(innen)'),
 Row(Note='Max. 3 Personen'),
 Row(Note='Max. 2 Personen'),
 Row(Note='4  Mitbewohner(innen)'),
 Row(Note='3  Mitbewohner(innen)'),
 Row(Note='Max. 1 Person'),
 Row(Note='6  Mitbewohner(innen)'),
 Row(Note='5  Mitbewohner(innen)'),
 Row(Note='2 Schlafzimmer'),
 Row(Note='10  Mitbewohner(innen)'),
 Row(Note='1 Schlafzimmer'),
 Row(Note=None)]

In [40]:
df.select('Provider Type').distinct().collect()

[Row(Provider Type='Studentenwohnungen'),
 Row(Provider Type='Co-living'),
 Row(Provider Type='Privater Vermieter'),
 Row(Provider Type='Vermietungsunternehmen')]

In [41]:
df.select('Address').distinct().collect()

[Row(Address='Boxhagener Strasse'),
 Row(Address='Donaustrasse'),
 Row(Address='Lehrter Strasse'),
 Row(Address='Ebertstrasse'),
 Row(Address='Martin-Luther-Strasse'),
 Row(Address='Scharnweberstrasse'),
 Row(Address='Leibnizstrasse'),
 Row(Address='Residenzstrasse'),
 Row(Address='Paul-Heyse-Strasse'),
 Row(Address='Karlsruher Strasse'),
 Row(Address='Wichertstrasse'),
 Row(Address='Voltairestrasse'),
 Row(Address='Otto-Franke-Strasse'),
 Row(Address='Mierendorffstrasse'),
 Row(Address='Germanenstrasse'),
 Row(Address='Alfred-Jung-Strasse'),
 Row(Address='Bismarckstrasse'),
 Row(Address='Grossbeerenstrasse'),
 Row(Address='Corinthstrasse'),
 Row(Address='Hechelstrasse'),
 Row(Address='Finowstrasse'),
 Row(Address='Hermannstrasse'),
 Row(Address='Mommsenstrasse'),
 Row(Address='Hagelberger Strasse'),
 Row(Address='Turmstrasse'),
 Row(Address='Luebecker Strasse'),
 Row(Address='Simon-Dach-Strasse'),
 Row(Address='Pappelallee'),
 Row(Address='Brueckenstrasse'),
 Row(Address='Klopstockstr

In [42]:
df.select('Rent Price Info').distinct().collect()

[Row(Rent Price Info='(einige Nebenkosten inklusive)'),
 Row(Rent Price Info='(Nebenkosten exklusive)'),
 Row(Rent Price Info='(Nebenkosten inklusive)')]

In [43]:
df.select('Listing Date').distinct().collect()

[Row(Listing Date='4. Juni'),
 Row(Listing Date='23. Aug.'),
 Row(Listing Date='9. Juli'),
 Row(Listing Date='16. Nov.'),
 Row(Listing Date='30. Okt.'),
 Row(Listing Date='29. Juli'),
 Row(Listing Date='8. Juli'),
 Row(Listing Date='20. Juni'),
 Row(Listing Date='28. Nov.'),
 Row(Listing Date='30. Sept.'),
 Row(Listing Date='16. Juni'),
 Row(Listing Date='3. Dez.'),
 Row(Listing Date='15. Juni'),
 Row(Listing Date='9. Juni'),
 Row(Listing Date='12. Juni'),
 Row(Listing Date='21. Juli'),
 Row(Listing Date='3. Juli'),
 Row(Listing Date='4. Juli'),
 Row(Listing Date='16. Juli'),
 Row(Listing Date='19. Juni'),
 Row(Listing Date='24. Juli'),
 Row(Listing Date='16. Aug.'),
 Row(Listing Date='1. Okt.'),
 Row(Listing Date='31. Aug.'),
 Row(Listing Date='10. Okt.'),
 Row(Listing Date='14. Juni'),
 Row(Listing Date='21. Sept.'),
 Row(Listing Date='22. Aug.'),
 Row(Listing Date='3. Aug.'),
 Row(Listing Date='5. Sept.'),
 Row(Listing Date='2. Aug.'),
 Row(Listing Date='16. Sept.'),
 Row(Listing Da

In [44]:
from pyspark.sql.functions import split
from pyspark.sql.functions import substring, length, expr, lit
from datetime import datetime

current_year = datetime.now().year

split_col = split(df['Listing Date'], ' ')
df = df.withColumn('Listing Day', regexp_replace(split_col[0], '\.', ''))
df = df.withColumn('Listing Month', split_col[1])
df = df.withColumn('Listing Year', lit(current_year))
df.select('Listing Day', 'Listing Month', 'Listing Year').show(n=20)

+-----------+-------------+------------+
|Listing Day|Listing Month|Listing Year|
+-----------+-------------+------------+
|          3|         Juni|        2024|
|          5|         Juni|        2024|
|          3|         Juni|        2024|
|          3|         Juni|        2024|
|          4|         Juni|        2024|
|         16|         Juni|        2024|
|          1|        Sept.|        2024|
|          3|         Juni|        2024|
|          3|         Juni|        2024|
|          3|         Juni|        2024|
|          3|         Juni|        2024|
|          3|         Juni|        2024|
|          3|         Juni|        2024|
|          3|         Juni|        2024|
|          3|         Juni|        2024|
|          5|         Juni|        2024|
|          1|         Aug.|        2024|
|          1|         Aug.|        2024|
|          1|         Aug.|        2024|
|          5|         Juli|        2024|
+-----------+-------------+------------+
only showing top

In [45]:
df.select('Listing Month').distinct().collect()

[Row(Listing Month='Juli'),
 Row(Listing Month='Nov.'),
 Row(Listing Month='Aug.'),
 Row(Listing Month='Sept.'),
 Row(Listing Month='Dez.'),
 Row(Listing Month='Okt.'),
 Row(Listing Month='Juni')]

In [46]:
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Jan.', '1'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Feb.', '2'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Marz', '3'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Apr.', '4'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Mai', '5'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Juni', '6'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Juli', '7'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Aug.', '8'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Sept.', '9'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Okt.', '10'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Nov.', '11'))
df = df.withColumn('Listing Month', regexp_replace('Listing Month', 'Dez.', '12'))

df.select('Listing Month').distinct().collect()

[Row(Listing Month='7'),
 Row(Listing Month='11'),
 Row(Listing Month='8'),
 Row(Listing Month='6'),
 Row(Listing Month='9'),
 Row(Listing Month='10'),
 Row(Listing Month='12')]

In [47]:
from pyspark.sql.functions import *

df = df.withColumn("Date",concat_ws("-",col("Listing Year"),col("Listing Month"),col("Listing Day")).cast("date"))
df.select('Date').distinct().collect()

[Row(Date=datetime.date(2024, 6, 4)),
 Row(Date=datetime.date(2024, 6, 12)),
 Row(Date=datetime.date(2024, 8, 27)),
 Row(Date=datetime.date(2024, 10, 24)),
 Row(Date=datetime.date(2024, 8, 30)),
 Row(Date=datetime.date(2024, 7, 8)),
 Row(Date=datetime.date(2024, 8, 5)),
 Row(Date=datetime.date(2024, 12, 1)),
 Row(Date=datetime.date(2024, 8, 12)),
 Row(Date=datetime.date(2024, 9, 30)),
 Row(Date=datetime.date(2024, 7, 5)),
 Row(Date=datetime.date(2024, 8, 14)),
 Row(Date=datetime.date(2024, 7, 11)),
 Row(Date=datetime.date(2024, 8, 4)),
 Row(Date=datetime.date(2024, 8, 7)),
 Row(Date=datetime.date(2024, 7, 24)),
 Row(Date=datetime.date(2024, 8, 1)),
 Row(Date=datetime.date(2024, 7, 17)),
 Row(Date=datetime.date(2024, 7, 30)),
 Row(Date=datetime.date(2024, 6, 15)),
 Row(Date=datetime.date(2024, 11, 5)),
 Row(Date=datetime.date(2024, 9, 3)),
 Row(Date=datetime.date(2024, 7, 21)),
 Row(Date=datetime.date(2024, 11, 14)),
 Row(Date=datetime.date(2024, 7, 25)),
 Row(Date=datetime.date(2024, 1